# Model

In [1]:
import os
import shutil
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image, ImageFile
import sys
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as mlines
import cv2
import csv
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors
from matplotlib.patches import ConnectionPatch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import CocoDetection
import argparse
import math
import torch.optim as optim

from torchvision.datasets import CIFAR10
from torchvision.datasets import VOCSegmentation
from sklearn.decomposition import PCA
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
# Check PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)

# If CUDA is available, print CUDA version
if cuda_available:
    print("CUDA version:", torch.version.cuda)

PyTorch version: 2.5.0+cpu
CUDA available: False


### Load the COCO Dataset

In [2]:
from utils import download_coco_dataset

# Define COCO path
coco_path = "datasets/COCO"
os.makedirs(coco_path, exist_ok=True)

# Download the dataset (if not already downloaded)
download_coco_dataset(coco_path)

KeyboardInterrupt: 

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
from pycocotools.coco import COCO
import requests
from PIL import ImageFile
import os

# HYPERPARAMETERS
BATCH_SIZE = 32
image_size = 224

# Select Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: ", device)

# Initialize the Model
model = PrivacyDINO(pca_components=16).to(device)
print(model)

# Enable loading truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Transformations
image_transforms = transforms.Compose([
    transforms.Resize((image_size, image_size), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
])

# Load COCO dataset
train_dataset = datasets.CocoDetection(
    root=f"{coco_data_path}/train2017",  # Path to train images
    annFile=f"{coco_data_path}/annotations/instances_train2017.json",  # Path to train annotations
    transform=image_transforms,  # Image transformations
)

val_dataset = datasets.CocoDetection(
    root=f"{coco_data_path}/val2017",  # Path to validation images
    annFile=f"{coco_data_path}/annotations/instances_val2017.json",  # Path to validation annotations
    transform=image_transforms,  # Image transformations
)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Check that the dataset is loaded
print(f"Total number of images in train set: {len(train_dataset)}")
print(f"Total number of images in validation set: {len(val_dataset)}")


device:  cuda


Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:01<00:00, 244MB/s]


PrivacyDINO(
  (original_model): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
    

In [ ]:
# Custom collate function to handle variable-sized annotations
def collate_fn(batch):
    images = []
    targets = []
    for img, target in batch:
        images.append(img)
        targets.append(target)  # Append annotations as they are (list of dicts)
    return torch.stack(images), targets

# Create DataLoaders with the custom collate function
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, collate_fn=collate_fn
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, collate_fn=collate_fn
)

# Check one sample from the DataLoader
sample = next(iter(train_loader))
images, targets = sample

print(f"Batch size: {len(images)}")
print(f"Image shape: {images[0].shape}")
print(f"Annotations for the first image: {targets[0]}")


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torchvision/datasets/coco.py", line 53, in __getitem__
    image = self._load_image(id)
  File "/usr/local/lib/python3.10/dist-packages/torchvision/datasets/coco.py", line 42, in _load_image
    return Image.open(os.path.join(self.root, path)).convert("RGB")
  File "/usr/local/lib/python3.10/dist-packages/PIL/Image.

In [ ]:
# Choose one index for visualization
idx = 0

# Forward pass through the model
x_tokens, x_cls, logits = model(sample.to(device))

# Show the output
print("x_tokens: ", x_tokens.shape)
print("x_cls: ", x_cls.shape)
print("logits: ", logits.shape)

# Convert the image and privacy map to numpy arrays
image_np = sample[idx].permute(1, 2, 0).cpu().detach().numpy()

# Create a figure and axes
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

# Display the original image
ax.imshow(image_np)
ax.set_title("Original Image")
ax.axis('off')
plt.show()


In [ ]:
def train(model, train_loader, num_epochs=10, learning_rate=1e-4, log_dir='./logs'):
    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # TensorBoard writer
    writer = SummaryWriter(log_dir)

    # Training loop
    model.train()

    for epoch in range(num_epochs):
        # Training phase
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        # Progress bar for each epoch
        with tqdm(train_loader, unit="batch") as tepoch:
            for batch_idx, (inputs, targets) in enumerate(tepoch):
                tepoch.set_description(f"Epoch {epoch + 1}/{num_epochs}")

                # Move inputs and targets to the device
                inputs, targets = inputs.to(device), targets.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                x_tokens, x_cls, logits = model(inputs)

                # Compute loss
                loss = criterion(logits, targets)

                # Backward pass and optimization
                loss.backward()
                optimizer.step()

                # Track statistics
                running_loss += loss.item()
                _, predicted = torch.max(logits, 1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                # Update progress bar
                accuracy = 100. * correct / total
                tepoch.set_postfix(loss=loss.item(), accuracy=accuracy)

                # Log the loss and accuracy to TensorBoard every batch
                writer.add_scalar('Training Loss', loss.item(), epoch * len(train_loader) + batch_idx)
                writer.add_scalar('Training Accuracy', accuracy, epoch * len(train_loader) + batch_idx)

        # Log the average loss and accuracy for each epoch
        avg_loss = running_loss / len(train_loader)
        epoch_accuracy = 100. * correct / total
        writer.add_scalar('Epoch Loss', avg_loss, epoch)
        writer.add_scalar('Epoch Accuracy', epoch_accuracy, epoch)

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():  # Disable gradient calculation for validation
            with tqdm(val_loader, unit="batch") as vepoch:
                for batch_idx, (inputs, targets) in enumerate(vepoch):
                    vepoch.set_description(f"Validation Epoch {epoch + 1}/{num_epochs}")

                    # Move inputs and targets to the device
                    inputs, targets = inputs.to(device), targets.to(device)

                    # Forward pass
                    x_tokens, x_cls, logits = model(inputs)

                    # Compute loss
                    loss = criterion(logits, targets)
                    val_loss += loss.item()

                    # Track statistics
                    _, predicted = torch.max(logits, 1)
                    val_total += targets.size(0)
                    val_correct += predicted.eq(targets).sum().item()

                    # Update progress bar
                    val_accuracy = 100. * val_correct / val_total
                    vepoch.set_postfix(val_loss=loss.item(), val_accuracy=val_accuracy)

        # Log validation loss and accuracy for each epoch
        avg_val_loss = val_loss / len(val_loader)
        val_epoch_accuracy = 100. * val_correct / val_total
        writer.add_scalar('Validation Loss', avg_val_loss, epoch)
        writer.add_scalar('Validation Accuracy', val_epoch_accuracy, epoch)

        print(f'Validation - Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_val_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%')

# Example call to train the model
train(model, train_loader, num_epochs=10, learning_rate=1e-2, log_dir='./logs/PrivacyDINO')
